In [29]:
import numpy

class matrix_factorization_learning_model():
    
    def __init__(self,R,a,b,it,k):
        self.R=R
        self.user_n, self.item_n=R.shape
        self.alpha=a
        self.beta=b
        self.iterations=it
        self.K=k
        
    def learn(self):
        self.b_users=numpy.zeros(self.user_n)
        self.b_items=numpy.zeros(self.item_n)
        self.b=numpy.mean(self.R[numpy.where(self.R!=0)])
        
        self.P=numpy.random.normal(scale=1./self.K,size=(self.user_n,self.K))
        self.Q=numpy.random.normal(scale=1./self.K,size=(self.item_n,self.K))
        rng=numpy.random.default_rng()
        
        self.samples=[]
        for i in range (self.user_n):
            for j in range (self.item_n):
                if (self.R[i][j])!=0:
                    self.samples.append((i,j,self.R[i][j]))

        for i in range (self.iterations):
            samplesize=min(100,len(self.samples))
            self.samplegroup=rng.choice(self.samples,samplesize,False)
            self.sgd()
        
            
    def leastCertain(self):
        Q=numpy.absolute(self.complete())
        return numpy.unravel_index(numpy.argmin(Q),Q.shape)

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.b_users[i]+self.b_items[j]+self.b+self.P[i,:].dot(self.Q[j,:].T)
            e = (r - prediction)
            
            self.b_users[i]+=self.alpha*(e-self.beta*self.b_users[i])
            self.b_items[j]+=self.alpha*(e-self.beta*self.b_items[j])
            
            self.P[i,:]+=self.alpha*(e*self.Q[j,:]-self.beta*self.P[i,:])
            self.Q[j,:]+=self.alpha*(e*self.P[i,:]-self.beta*self.Q[j,:])
            
    def get_rating(self,i,j):
        return self.b_users[i]+self.b_items[j]+self.b+self.P[i,:].dot(self.Q[j,:].T)
    
    def complete(self):
        return self.P.dot(self.Q.T)+self.b+self.b_users[:,numpy.newaxis]+self.b_items[numpy.newaxis,:]
    
def voting(C):
    D=numpy.zeros(shape=C[0].R.shape)
    for i in range (D.shape[0]):
        for j in range (D.shape[1]):
            vn=0
            vp=0
            for item in C:
                if (item.complete()[i][j]>0):
                    vp+=1
                else:
                    vn+=1
            if (vp==0 or vn==0):
                s1=0
                s2=0
            else:
                s1=vp/len(C)*numpy.log(vp/len(C))
                s2=vn/len(C)*numpy.log(vn/len(C))
            D[i][j]=-s1-s2

    return numpy.unravel_index(numpy.argmax(D),D.shape)
    
def twoWayVoting(C):
    c1=C[0].complete()
    c2=C[1].complete()
    D=numpy.absolute(numpy.subtract(c1,c2))
    print(numpy.unravel_index(numpy.argmax(D),D.shape))
    return numpy.amax(D)
    
        
        

In [30]:
M = numpy.array([
    [1, -1, 1, 0],
    [1, 1, 0, 0],
    [0, 0, 1, 1],
    [1, 1, 0, -1],
    [0, 1, 0, 0],
])
print(M)
print()
C=[]
print("Least Certain Instance's Indexes by models: ")
for i in range(5):
    model = matrix_factorization_learning_model(M, 0.1, 0.01, 25, 2)
    model.learn()
    print(model.leastCertain())
    C.append(model)
for i in range(5):
    model = matrix_factorization_learning_model(M, 0.1, 0.01, 15, 2)
    model.learn()
    print(model.leastCertain())
    C.append(model)

print("\nIndex of the least agreed instance by committee:")
d=voting(C)
print(d)
C=numpy.reshape(C,(2,5))
print("\nBiggest differnce in two models:")
twoWayVoting(C.reshape(10,order='F'))





[[ 1 -1  1  0]
 [ 1  1  0  0]
 [ 0  0  1  1]
 [ 1  1  0 -1]
 [ 0  1  0  0]]

Least Certain Instance's Indexes by models: 
(1, 3)
(4, 2)
(2, 1)
(0, 3)
(4, 3)
(1, 3)
(1, 3)
(4, 3)
(4, 0)
(4, 3)

Index of the least agreed instance by committee:
(0, 3)

Biggest differnce in two models:
(0, 3)


1.8828491466661474